# One iteration of Arnoldi's algorithm

In [9]:
import numpy as np

In [10]:
def arnoldi_one_iter(A, V, k, tol = 1e-12):
    """
    Computes the new vectors of the Arnoldi iteration for both V_{k+1} and H_{k + 1, k}

    Parameters:
    -----------
    A : numpy.ndarray
        An (n x n) array representing the matrix.
          
    V : numpy.ndarray
        An (n x (k + 1)) array. The current Krylov orthonormal basis. 
        The k + 1 column is zero and it will be set to v_new in the gmres.
      
    k : int
        One less than the step we are obtaining in the Arnoldi's algorithm to increase
        the dimension of the Krylov subspace. Must be >= 0.
    
    tol : float, optional
        Tolerance for convergence. Default is 1e-12.

    Returns:
    --------
    h_k : numpy.ndarray
        The column k + 1 in Hessenberg matrix (k starts at 0).

    v_new : numpy.ndarray
        The new orthogonal vector in the basis of the Krylov subspace.

    """

    # Initialize k + 2 nonzero elements of H along column k
    h_k = np.zeros(k + 2)

    # Calculate the new vector in the Krylov subspace
    v_new = A @ V[:, k]

    # Calculate the first k elements of the k-th Hessenberg column
    for j in range(k + 1):
        h_k[j] = v_new @ V[:, j]
        v_new -= h_k[j] * V[:, j]

    # Add the k+1 element
    h_k[k + 1] = np.linalg.norm(v_new)

    if h_k[k + 1] <= tol:
        # Early termination with exact solution
        return h_k, None
    
    else:
        # Find the new orthogonal vector in the basis of the Krylov subspace
        v_new /= h_k[k + 1]

    return h_k, v_new

# Unit tests

### Small matrix just to verify the algorithm is working

#### Verifying the elements from the first two iteration in the GMRES are correct

In [11]:
n = 4
A = np.array([[1, 1, 4, 9], [3, 4, 6, 9], [4, 1, 1, 3], [3, 2, 1, 1]])
b = np.array([3, 2, 2, -3])
x0 = np.array([0, 0, 0, 0])
r0 = b - A @ x0

V = np.zeros((n, 1))
beta = np.linalg.norm(r0)
V[:, 0] = r0 / beta
V = np.concatenate((V, np.zeros((n, 1))), axis=1)

H = np.zeros((n + 1, 1))
H = np.concatenate((H, np.zeros((n + 1, 1))), axis=1)
k = 0

H[:(k + 2), k], v_new  = arnoldi_one_iter(A, V, k)
V[:, k + 1] = v_new

In [12]:
V, H

(array([[ 0.58834841, -0.44357572],
        [ 0.39223227,  0.41464687],
        [ 0.39223227,  0.72804276],
        [-0.58834841,  0.31821737]]),
 array([[-2.30769231,  0.        ],
        [ 3.12888811,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ]]))

In [13]:
V[:, : k + 1] , H[: k + 2, : k + 1]

(array([[ 0.58834841],
        [ 0.39223227],
        [ 0.39223227],
        [-0.58834841]]),
 array([[-2.30769231],
        [ 3.12888811]]))

In [14]:
from numpy.linalg import qr
Q, R = qr(H[: k + 2, : k + 1], mode = 'complete')
R[:-1, :], Q[0][:-1]

(array([[3.8878509]]), array([-0.59356502]))

#### Another iteration

In [15]:
# Arnoldi iteration
V = np.concatenate((V, np.zeros((n, 1))), axis=1)
H = np.concatenate((H, np.zeros((n + 1, 1))), axis=1)
V, H

(array([[ 0.58834841, -0.44357572,  0.        ],
        [ 0.39223227,  0.41464687,  0.        ],
        [ 0.39223227,  0.72804276,  0.        ],
        [-0.58834841,  0.31821737,  0.        ]]),
 array([[-2.30769231,  0.        ,  0.        ],
        [ 3.12888811,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ]]))

In [16]:
k = 1

H[:(k + 2), k], v_new  = arnoldi_one_iter(A, V, k)
V, H

(array([[ 0.58834841, -0.44357572,  0.        ],
        [ 0.39223227,  0.41464687,  0.        ],
        [ 0.39223227,  0.72804276,  0.        ],
        [-0.58834841,  0.31821737,  0.        ]]),
 array([[-2.30769231,  6.15281806,  0.        ],
        [ 3.12888811,  0.99400237,  0.        ],
        [ 0.        ,  7.19312005,  0.        ],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ]]))

In [17]:
V[:, : k + 1] , H[: k + 2, : k + 1]

(array([[ 0.58834841, -0.44357572],
        [ 0.39223227,  0.41464687],
        [ 0.39223227,  0.72804276],
        [-0.58834841,  0.31821737]]),
 array([[-2.30769231,  6.15281806],
        [ 3.12888811,  0.99400237],
        [ 0.        ,  7.19312005]]))

In [18]:
V[:, k + 1] = v_new
V

array([[ 0.58834841, -0.44357572,  0.35702353],
       [ 0.39223227,  0.41464687,  0.65820942],
       [ 0.39223227,  0.72804276, -0.39120314],
       [-0.58834841,  0.31821737,  0.53502771]])

In [19]:
V[:, :k + 1], V[:, :-1]

(array([[ 0.58834841, -0.44357572],
        [ 0.39223227,  0.41464687],
        [ 0.39223227,  0.72804276],
        [-0.58834841,  0.31821737]]),
 array([[ 0.58834841, -0.44357572],
        [ 0.39223227,  0.41464687],
        [ 0.39223227,  0.72804276],
        [-0.58834841,  0.31821737]]))

In [20]:
Q, R = qr(H[: k + 2, : k + 1], mode = 'complete')
Q[0][:-1]
Q,R

(array([[-0.59356502, -0.49116197, -0.63752685],
        [ 0.80478604, -0.36225351, -0.47020403],
        [ 0.        , -0.79216937,  0.61030131]]),
 array([[ 3.8878509 , -2.85213836],
        [ 0.        , -9.08028048],
        [ 0.        ,  0.        ]]))